# **Write a program to demonstrate the working of the decision tree based ID3 algorithm. Use an appropriate data set for building the decision tree and apply this knowledge to classify a new sample.** #

In [2]:
import pandas as pd
import numpy as np

# Import the dataset and define the feature as well as the target columns
dataset = pd.read_csv('PlayTennis.csv', names=['outlook', 'temperature', 'humidity', 'wind', 'class'])

# Define the attributes (excluding target 'class')
attributes = ['Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis']

# Function to calculate the entropy of a dataset
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter is the target_col, which specifies the target column.
    """
    elements, counts = np.unique(target_col, return_counts=True)
    entropy = np.sum([(-counts[i] / np.sum(counts)) * np.log2(counts[i] / np.sum(counts)) for i in range(len(elements))])
    return entropy

# Function to calculate the Information Gain
def InfoGain(data, split_attribute_name, target_name="class"):
    """
    Calculate the information gain for a specific attribute.
    """
    # Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])

    # Calculate the values and corresponding counts for the split attribute
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)

    # Calculate the weighted entropy
    weighted_entropy = np.sum([(counts[i] / np.sum(counts)) * 
                               entropy(data.where(data[split_attribute_name] == vals[i]).dropna()[target_name]) 
                               for i in range(len(vals))])

    # Calculate the information gain
    information_gain = total_entropy - weighted_entropy
    return information_gain

# Function to implement the ID3 algorithm
def ID3(data, originaldata, features, target_attribute_name="class", parent_node_class=None):
    """
    Implement the ID3 algorithm to build a decision tree.
    """
    # Stopping criteria:
    
    # If all target values have the same value, return that value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]

    # If the dataset is empty, return the mode target feature value of the original dataset
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name], return_counts=True)[1])]

    # If there are no more features to split on, return the parent node class
    elif len(features) == 0:
        return parent_node_class

    else:
        # Set the default value for this node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]

        # Select the feature with the best information gain
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        # Create the tree structure (root node)
        tree = {best_feature: {}}

        # Remove the best feature from the feature set
        features = [i for i in features if i != best_feature]

        # Grow a branch for each possible value of the feature
        for value in np.unique(data[best_feature]):
            sub_data = data.where(data[best_feature] == value).dropna()

            # Call ID3 algorithm recursively for each subset of the dataset
            subtree = ID3(sub_data, originaldata, features, target_attribute_name, parent_node_class)

            # Add the subtree to the root node
            tree[best_feature][value] = subtree

        return tree

# Function to predict the class label of a query using the decision tree
def predict(query, tree, default=1):
    """
    Predict the class label for a given query instance using the decision tree.
    """
    for key in query.keys():
        if key in tree.keys():
            try:
                result = tree[key][query[key]]
            except:
                return default

            # Recursively predict if result is a subtree
            if isinstance(result, dict):
                return predict(query, result)
            else:
                return result

# Function to split the dataset into training data
def train_test_split(dataset):
    """
    Split the dataset into training and testing data.
    """
    training_data = dataset.iloc[:14].reset_index(drop=True)
    return training_data

# Function to test the prediction accuracy of the decision tree
def test(data, tree):
    """
    Test the decision tree's accuracy on the dataset.
    """
    # Create query instances (remove the target column and convert to dictionary format)
    queries = data.iloc[:, :-1].to_dict(orient="records")

    # Create a DataFrame to store predictions
    predicted = pd.DataFrame(columns=["predicted"])

    # Calculate predictions and accuracy
    for i in range(len(data)):
        predicted.loc[i, "predicted"] = predict(queries[i], tree, 1.0)

    accuracy = (np.sum(predicted["predicted"] == data["class"]) / len(data)) * 100
    print('The prediction accuracy is:', accuracy, '%')

# Train the decision tree, display it, and test its accuracy
training_data = train_test_split(dataset)
tree = ID3(training_data, training_data, training_data.columns[:-1])
print('Decision Tree:', tree)
test(training_data, tree)


Decision Tree: {'outlook': {'Outlook': 'Play Tennis', 'Overcast': 'Yes', 'Rain': {'wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}
The prediction accuracy is: 100.0 %
